# bigram / bow / no filtered


In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from enum import Enum
from pprint import pprint
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import Phrases

import pyLDAvis
import pyLDAvis.gensim
from gensim.models.ldamodel import LdaModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.DEBUG)

2019-11-05 01:08:36,134 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## 自定義 data types and functions

In [2]:
class ContentType(Enum):
    TIT = 'title'
    ABS = 'abstract'
    AUT = 'author'
    SEC = 'section'
    
def get_contents(content_type):
    all_contents = []
    dataset_path = '../dataset'
    for file in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, file)
        if os.path.isfile(file_path):
            with open(file_path) as f:  
                line = f.readlines()
                if content_type == ContentType.AUT:
                    line = line[1]
                elif content_type == ContentType.SEC:
                    line = line[2]
                elif content_type == ContentType.ABS:
                    line = line[3]
                else:
                    line = line[0]
                line = line.strip()
                all_contents.append(line)
        else:
            print(file_path + ' does not exist.')
    return all_contents


def get_all_titles():
    return get_contents(ContentType.TIT)

def get_all_authors():        
    return get_contents(ContentType.AUT)

def get_all_sections():
    return get_contents(ContentType.SEC)

def get_all_abstracts():
    return get_contents(ContentType.ABS)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = wordnet_lemmatizer.lemmatize(token, pos='v')
            token = wordnet_lemmatizer.lemmatize(token, pos='n')
            result.append(token)
    return result

### 取出所有摘要

In [3]:
contents = get_all_abstracts()
print('共',len(contents),'篇論文\n')

documents = pd.DataFrame(data=contents,columns=['abstract'])
documents['index'] = documents.index
documents[:10]

共 1343 篇論文



abstract  index
0  We consider the problem of actively eliciting ...      0
1  We investigate the task of distractor generati...      1
2  The most common representation formalisms for ...      2
3  Statistical relational learning models are pow...      3
4  Multimodal representation learning is gaining ...      4
5  Reinforcement learning (RL) has shown its adva...      5
6  Selecting appropriate tutoring help actions th...      6
7  Recognizing time expressions is a fundamental ...      7
8  When facing large-scale image datasets, online...      8
9  Temporal modeling in videos is a fundamental y...      9

### 預處理的全部論文摘要

In [4]:
processed_docs = documents['abstract'].map(preprocess)
processed_docs[:10]

bigram = Phrases(processed_docs, min_count=10)

for idx in range(len(processed_docs)):
    for token in bigram[processed_docs[idx]]:
        if '_' in token:
            processed_docs[idx].append(token)

2019-11-05 01:08:49,475 : INFO : collecting all words and their counts
2019-11-05 01:08:49,477 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-11-05 01:08:49,671 : INFO : collected 89298 word types from a corpus of 121739 words (unigram + bigrams) and 1343 sentences
2019-11-05 01:08:49,672 : INFO : using 89298 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>


## Dataset

### 產生字典

In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('共',len(dictionary),'個字\n')

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

dictionary.save('../corpus/dict_bigram_filtered.dict')
#loaded_dict = corpora.Dictionary.load('../corpus/mydict.dict')

2019-11-05 01:09:17,202 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-05 01:09:17,348 : INFO : built Dictionary(7141 unique tokens: ['active', 'active_learn', 'actively', 'adaptive', 'aggregation']...) from 1343 documents (total 128091 corpus positions)
2019-11-05 01:09:17,360 : INFO : discarding 5583 tokens: [('actively', 9), ('algorithm', 358), ('approach', 483), ('coefficient', 8), ('collective', 7), ('elicitation', 3), ('equity', 2), ('gini', 1), ('interleave', 3), ('knapsack', 5)]...
2019-11-05 01:09:17,360 : INFO : keeping 1558 tokens which were in no less than 10 and no more than 268 (=20.0%) documents
2019-11-05 01:09:17,363 : DEBUG : rebuilding dictionary, shrinking gaps
2019-11-05 01:09:17,365 : INFO : resulting dictionary: Dictionary(1558 unique tokens: ['active', 'active_learn', 'adaptive', 'aggregation', 'allow']...)
2019-11-05 01:09:17,367 : INFO : saving Dictionary object under ../corpus/dict_bigram_filtered.dict, separately None
2019-11-05 01:09

共 7141 個字

Number of unique words after removing rare and common words: 1558


### 產生 bag of words corpus

In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('共',len(bow_corpus),'筆')

corpora.MmCorpus.serialize('../corpus/corpus_bigram_filtered.mm', bow_corpus)
#corpus_test = corpora.MmCorpus('../corpus/bow_corpus.mm')

2019-11-05 01:09:37,306 : INFO : storing corpus in Matrix Market format to ../corpus/corpus_bigram_filtered.mm
2019-11-05 01:09:37,307 : WARNING : this function is deprecated, use smart_open.open instead
/Users/alexis/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
2019-11-05 01:09:37,308 : DEBUG : {'uri': '../corpus/corpus_bigram_filtered.mm', 'mode': 'wb+', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': {}}
2019-11-05 01:09:37,309 : INFO : saving sparse matrix to ../corpus/corpus_bigram_filtered.mm
2019-11-05 01:09:37,312 : INFO : PROGRESS: saving document #0
2019-11-05 01:09:37,408 : INFO : PROGRESS: saving document #1000
2019-11-05 01:09:37,431 : INFO : saved 1343x1558 matrix, density=3.011% (

共 1343 筆


## Train LDA models

In [7]:
num_topics = 10
num_words = 10

### Running LDA using BOW

In [8]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=num_topics, 
                                       id2word=dictionary, 
                                       passes=30,
                                       iterations=300,
                                       eval_every=1)

2019-11-05 01:10:19,813 : INFO : using symmetric alpha at 0.1
2019-11-05 01:10:19,815 : INFO : using symmetric eta at 0.1
2019-11-05 01:10:19,819 : INFO : using serial LDA version on this node
2019-11-05 01:10:19,827 : INFO : running online LDA training, 10 topics, 30 passes over the supplied corpus of 1343 documents, updating every 6000 documents, evaluating every ~1343 documents, iterating 300x with a convergence threshold of 0.001000
2019-11-05 01:10:19,828 : INFO : training LDA model using 3 processes
2019-11-05 01:10:19,843 : DEBUG : worker process entering E-step loop
2019-11-05 01:10:19,855 : DEBUG : worker process entering E-step loop
2019-11-05 01:10:19,865 : DEBUG : worker process entering E-step loop
2019-11-05 01:10:19,863 : DEBUG : getting a new job
2019-11-05 01:10:19,877 : DEBUG : getting a new job
2019-11-05 01:10:19,878 : DEBUG : getting a new job
2019-11-05 01:10:19,900 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size

2019-11-05 01:10:29,220 : INFO : topic #3 (0.100): 0.012*"label" + 0.009*"sample" + 0.007*"structure" + 0.007*"classification" + 0.007*"graph" + 0.006*"interaction" + 0.006*"study" + 0.006*"loss" + 0.005*"question" + 0.005*"deep"
2019-11-05 01:10:29,209 : DEBUG : getting a new job
2019-11-05 01:10:29,223 : INFO : topic diff=0.250896, rho=0.462671
2019-11-05 01:10:29,233 : DEBUG : bound: at document #0
2019-11-05 01:10:30,312 : INFO : -7.101 per-word bound, 137.3 perplexity estimate based on a held-out corpus of 1343 documents with 87020 words
2019-11-05 01:10:30,312 : INFO : PROGRESS: pass 4, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-05 01:10:30,338 : DEBUG : processing chunk #0 of 1343 documents
2019-11-05 01:10:30,370 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-05 01:10:31,220 : DEBUG : 1341/1343 documents converged within 300 iterations
2019-11-05 01:10:31,224 : DEBUG : processed chunk, queuing the result
2019-11-05 01

2019-11-05 01:10:36,678 : INFO : topic diff=0.203510, rho=0.339588
2019-11-05 01:10:36,690 : DEBUG : bound: at document #0
2019-11-05 01:10:37,627 : INFO : -7.026 per-word bound, 130.4 perplexity estimate based on a held-out corpus of 1343 documents with 87020 words
2019-11-05 01:10:37,628 : INFO : PROGRESS: pass 8, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-05 01:10:37,657 : DEBUG : processing chunk #0 of 1343 documents
2019-11-05 01:10:37,659 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-05 01:10:38,341 : DEBUG : 1343/1343 documents converged within 300 iterations
2019-11-05 01:10:38,347 : DEBUG : processed chunk, queuing the result
2019-11-05 01:10:38,353 : DEBUG : updating topics
2019-11-05 01:10:38,351 : DEBUG : result put
2019-11-05 01:10:38,355 : DEBUG : getting a new job
2019-11-05 01:10:38,357 : INFO : topic #0 (0.100): 0.019*"agent" + 0.019*"game" + 0.009*"strategy" + 0.008*"mechanism" + 0.007*"machine" + 0.007*"qu

2019-11-05 01:10:44,206 : INFO : PROGRESS: pass 12, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-05 01:10:44,230 : DEBUG : processing chunk #0 of 1343 documents
2019-11-05 01:10:44,232 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-05 01:10:44,858 : DEBUG : 1342/1343 documents converged within 300 iterations
2019-11-05 01:10:44,863 : DEBUG : processed chunk, queuing the result
2019-11-05 01:10:44,870 : DEBUG : updating topics
2019-11-05 01:10:44,868 : DEBUG : result put
2019-11-05 01:10:44,874 : INFO : topic #9 (0.100): 0.040*"label" + 0.017*"answer" + 0.012*"question" + 0.011*"instance" + 0.011*"class" + 0.011*"classification" + 0.010*"query" + 0.009*"multi" + 0.008*"attention" + 0.008*"dataset"
2019-11-05 01:10:44,872 : DEBUG : getting a new job
2019-11-05 01:10:44,875 : INFO : topic #6 (0.100): 0.039*"domain" + 0.016*"target" + 0.013*"source" + 0.011*"distribution" + 0.010*"transfer" + 0.010*"adaptation" + 0.009*"representat

2019-11-05 01:10:50,464 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-05 01:10:51,067 : DEBUG : 1342/1343 documents converged within 300 iterations
2019-11-05 01:10:51,073 : DEBUG : processed chunk, queuing the result
2019-11-05 01:10:51,079 : DEBUG : updating topics
2019-11-05 01:10:51,077 : DEBUG : result put
2019-11-05 01:10:51,082 : INFO : topic #8 (0.100): 0.020*"user" + 0.017*"sequence" + 0.017*"attention" + 0.014*"text" + 0.013*"word" + 0.011*"recurrent" + 0.010*"temporal" + 0.010*"generate" + 0.009*"prediction" + 0.009*"context"
2019-11-05 01:10:51,086 : INFO : topic #0 (0.100): 0.020*"agent" + 0.020*"game" + 0.010*"strategy" + 0.009*"machine" + 0.008*"mechanism" + 0.008*"study" + 0.007*"human" + 0.006*"question" + 0.006*"equilibrium" + 0.006*"translation"
2019-11-05 01:10:51,088 : DEBUG : getting a new job
2019-11-05 01:10:51,088 : INFO : topic #7 (0.100): 0.033*"video" + 0.032*"agent" + 0.029*"action" + 0.024*"plan" + 0.018*"temporal" + 0.015*"frame" + 0

2019-11-05 01:10:57,142 : DEBUG : processed chunk, queuing the result
2019-11-05 01:10:57,148 : DEBUG : updating topics
2019-11-05 01:10:57,145 : DEBUG : result put
2019-11-05 01:10:57,152 : INFO : topic #3 (0.100): 0.011*"structure" + 0.011*"graph" + 0.010*"sample" + 0.008*"accuracy" + 0.007*"hash" + 0.007*"machine" + 0.006*"loss" + 0.006*"distance" + 0.006*"cluster" + 0.006*"deep"
2019-11-05 01:10:57,154 : INFO : topic #2 (0.100): 0.023*"view" + 0.019*"object" + 0.018*"cluster" + 0.014*"multi" + 0.014*"user" + 0.013*"item" + 0.008*"multi_view" + 0.008*"student" + 0.008*"story" + 0.007*"detection"
2019-11-05 01:10:57,150 : DEBUG : getting a new job
2019-11-05 01:10:57,156 : INFO : topic #0 (0.100): 0.020*"game" + 0.020*"agent" + 0.011*"strategy" + 0.009*"machine" + 0.008*"mechanism" + 0.008*"study" + 0.007*"human" + 0.007*"decision" + 0.006*"equilibrium" + 0.006*"consider"
2019-11-05 01:10:57,158 : INFO : topic #4 (0.100): 0.011*"function" + 0.010*"number" + 0.010*"optimization" + 0.0

2019-11-05 01:11:03,139 : INFO : topic #1 (0.100): 0.039*"image" + 0.017*"deep" + 0.012*"adversarial" + 0.010*"layer" + 0.009*"object" + 0.009*"convolutional" + 0.008*"generate" + 0.008*"level" + 0.008*"region" + 0.008*"detection"
2019-11-05 01:11:03,136 : DEBUG : getting a new job
2019-11-05 01:11:03,141 : INFO : topic #3 (0.100): 0.012*"structure" + 0.011*"graph" + 0.010*"sample" + 0.009*"accuracy" + 0.007*"machine" + 0.007*"distance" + 0.007*"hash" + 0.007*"cluster" + 0.007*"loss" + 0.006*"study"
2019-11-05 01:11:03,143 : INFO : topic #9 (0.100): 0.043*"label" + 0.020*"answer" + 0.017*"question" + 0.012*"class" + 0.012*"instance" + 0.012*"multi" + 0.012*"query" + 0.012*"classification" + 0.009*"dataset" + 0.008*"supervise"
2019-11-05 01:11:03,145 : INFO : topic #8 (0.100): 0.019*"user" + 0.019*"sequence" + 0.018*"attention" + 0.016*"text" + 0.014*"word" + 0.012*"recurrent" + 0.011*"temporal" + 0.011*"generate" + 0.010*"prediction" + 0.009*"context"
2019-11-05 01:11:03,148 : INFO : t

2019-11-05 01:11:08,966 : DEBUG : getting a new job
2019-11-05 01:11:08,970 : INFO : topic #7 (0.100): 0.035*"video" + 0.034*"agent" + 0.032*"action" + 0.026*"plan" + 0.019*"temporal" + 0.016*"frame" + 0.015*"policy" + 0.013*"reinforcement" + 0.010*"reinforcement_learn" + 0.007*"environment"
2019-11-05 01:11:08,974 : INFO : topic #5 (0.100): 0.022*"knowledge" + 0.021*"graph" + 0.018*"sentence" + 0.018*"language" + 0.018*"word" + 0.016*"relation" + 0.014*"semantic" + 0.012*"embed" + 0.011*"entity" + 0.011*"natural"
2019-11-05 01:11:08,976 : INFO : topic #4 (0.100): 0.012*"function" + 0.010*"number" + 0.010*"optimization" + 0.009*"sample" + 0.008*"solution" + 0.007*"complexity" + 0.006*"efficient" + 0.006*"high" + 0.006*"gradient" + 0.006*"solve"
2019-11-05 01:11:08,978 : INFO : topic diff=0.056616, rho=0.183582
2019-11-05 01:11:08,984 : DEBUG : bound: at document #0
2019-11-05 01:11:09,800 : INFO : -6.920 per-word bound, 121.1 perplexity estimate based on a held-out corpus of 1343 docum

In [9]:
vis_data = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
# 在 notebook 中显示可视化结果，需要调用 display 方法，或者执行 “pyLDAvis.enable_notebook()” ，即可在 notebook 中自动展示可视化结果，无需再调用 display
pyLDAvis.display(vis_data)

2019-11-05 01:11:19,192 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-05 01:11:19,732 : DEBUG : 1343/1343 documents converged within 300 iterations
/Users/alexis/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [10]:
file_name = '../models/lda_bigram_bow_filtered_topic_'+ str(num_topics) +'.model'
print(file_name)
lda_model.save(file_name)

2019-11-05 01:12:28,237 : INFO : saving LdaState object under ../models/lda_bigram_bow_filtered_topic_10.model.state, separately None
2019-11-05 01:12:28,241 : WARNING : this function is deprecated, use smart_open.open instead
/Users/alexis/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:160: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  args, varargs, keywords, defaults = inspect.getargspec(kallable)
2019-11-05 01:12:28,242 : DEBUG : {'uri': '../models/lda_bigram_bow_filtered_topic_10.model.state', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': {}}
2019-11-05 01:12:28,245 : INFO : saved ../models/lda_bigram_bow_filtered_topic_10.model.state
2019-11-05 01:12:28,250 : WARNING : this function is deprecated, use smart_open.open instead
/Users/alexis/anaconda3/lib/python3.7/site-packages/smart_open/sm

../models/lda_bigram_bow_filtered_topic_10.model
